# 全部模型比较


## 配置通用参数

In [12]:
import sys
import importlib
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, Flatten, Dropout, BatchNormalization, Activation
from keras.layers import LSTM, Embedding
from keras.models import Sequential
from keras.regularizers import l2
from keras.utils import plot_model
import os
importlib.reload(sys)
root_path = os.path.split(sys.path[0])[0]
# 词向量模型
VECTOR_DIR = root_path+'/sgns/baike.vectors.bin'
# 词向量模型的维度
EMBEDDING_DIM = 200
# 截取文本最大长度
MAX_SEQUENCE_LENGTH = 150
# 验证集与测试集在总文本的比例
VALIDATION_SPLIT = 0.40
# 测试集的个数
TEST_NUM = 900

## 获取事件类数据集

In [9]:
import numpy as np
test_texts = open(root_path+'/data/incident_rumor/virus_test.txt', encoding='utf-8').read().split('\n')
test_labels = open(root_path+'/data/incident_rumor/label_virus_test.txt', encoding='utf-8').read().split('\n')
train_texts = open(root_path+'/data/incident_rumor/news&incident_rumor.txt', encoding='utf-8').read().split('\n')
train_labels = open(root_path+'/data/incident_rumor/label_incident_rumor.txt', encoding='utf-8').read().split('\n')
# np.random.seed(100)
# np.random.shuffle(train_texts)
# np.random.seed(100)
# np.random.shuffle(train_labels)
all_labels = train_labels + test_labels
all_texts = train_texts + test_texts

## 获取precision recall与f1对象

In [10]:
import keras_metrics as km
precision = km.categorical_precision(label=1)
# Calculate recall for the first label.
recall = km.categorical_recall(label=1)
f1 = km.categorical_f1_score(label=1)

tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> tp
tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> fp
tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> tp
tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> fn
tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> tp
tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> fp
tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> tp
tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> fn


## 读取词向量模型

In [13]:
import gensim
w2v_model = gensim.models.KeyedVectors.load_word2vec_format(VECTOR_DIR, binary=True)

## LSTM+word2vec

In [14]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_texts)
sequences = tokenizer.texts_to_sequences(all_texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = to_categorical(np.asarray(all_labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

print ('(3) split data set...')
p1 = int(len(data)*(1 - VALIDATION_SPLIT))
x_train = data[:p1]
y_train = labels[:p1]
x_val = data[p1:len(data)-TEST_NUM]
y_val = labels[p1:len(labels)-TEST_NUM]
x_test = data[len(data)-TEST_NUM:]
y_test = labels[len(labels)-TEST_NUM:]
print ('train docs: '+str(len(x_train)))
print ('val docs: '+str(len(x_val)))
print ('test docs: '+str(len(x_test)))

print ('(4) load word2vec as embedding...')
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
not_in_model = 0
in_model = 0
for word, i in word_index.items():
    if str(word) in w2v_model:
        in_model += 1
        embedding_matrix[i] = np.asarray(w2v_model[str(word)], dtype='float32')
    else:
        not_in_model += 1
print (str(not_in_model)+' words not in w2v model')

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

print ('(5) training model...')

model = Sequential()
model.add(embedding_layer)
model.add(LSTM(256, dropout=0.5, recurrent_dropout=0.2,W_regularizer=l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(labels.shape[1], activation='softmax'))
model.summary()
plot_model(model, to_file='model.png',show_shapes=True)
model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['acc',precision, recall,f1])
print (model.metrics_names)
history = model.fit(x_train, y_train, validation_data=(x_val, y_val),validation_split=0.33, epochs=12, batch_size=128,shuffle=True)
print ('(6) testing model...')
print (model.evaluate(x_test,y_test))

Found 30798 unique tokens.
Shape of data tensor: (4642, 150)
Shape of label tensor: (4642, 2)
(3) split data set...
train docs: 2785
val docs: 957
test docs: 900
(4) load word2vec as embedding...
7018 words not in w2v model
(5) training model...


c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:45: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(256, dropout=0.5, recurrent_dropout=0.2, kernel_regularizer=<keras.reg...)`


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 200)          6159800   
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               467968    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 514       
Total params: 6,628,282
Trainable params: 468,482
Non-trainable params: 6,159,800
_________________________________________________________________
['loss', 'acc', 'precision', 'recall', 'f1_score']
Train on 2785 samples, validate on 957 samples
Epoch 1/12
2785/2785 [==============================] - 4s 1ms/step - loss: 0.6813 - acc: 0.8302 - precision: 0.7812 - recall: 0.8263 - f1